## Simple linear regression

Firstly, we will import the required libraries and load the `Advertising` dataset.

In [ ]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api import ols

%matplotlib inline
plt.style.use("seaborn-white")

**Load Datasets**

Datasets available on https://www.statlearning.com/resources-first-edition

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Advertising.csv"

advertising_df = pd.read_csv(
    data_url, header=0, index_col=0
)  # read Boston data as pandas data frame
advertising_df.info()

In [ ]:
advertising_df.head()

Simple linear regression assumes that there is a approximately a linear relationship between a quantitative response $Y$ and a single predictor $X$. Mathematically, the linear relationship can be expressed as

\begin{equation}
Y \approx \beta_0 + \beta_1 X
\end{equation}

where $\beta_0$ and $\beta_1$ are two unknown constants that represent the *intercept* and *slope* of the linear model. Together, $\beta_0$ and $\beta_1$ are called the *model coefficients*, or *parameters*. We can describe the linear relationship as *regressing* $Y$ onto $X$. For example, $X$ may represent `TV` advertising and $Y$ may represent `sales`. Then we can regress sales onto TV by fitting the model

\begin{equation}
\textrm{sales} \approx \beta_0 + \beta_1 \times \textrm{TV}
\end{equation}



### Estimating the coefficients

The goal of simple linear regression is to estimate the unknown parameters $\beta_0$ and $\beta_1$ from the data. The estimated values of $\beta_0$ and $\beta_1$ are denoted by $\hat{\beta}_0$ and $\hat{\beta}_1$ respectively. The estimated values of $\hat{\beta}_0$ and $\hat{\beta}_1$ are obtained by minimizing the sum of squared residuals (SSR) or the sum of squared errors (SSE). The SSR is defined as

### Figure 3.1 - Least squares fit

In [ ]:
sns.regplot(
    x=advertising_df.TV,
    y=advertising_df.Sales,
    order=1,
    ci=None,
    scatter_kws={"color": "r", "s": 9},
)
plt.xlim(-10, 310)
plt.ylim(ymin=0)
plt.show()

### Figure 3.2 - Regression coefficients - RSS
Note that the text in the book describes the coefficients based on uncentered data, whereas the plot shows the model based on centered data. The latter is visually more appealing for explaining the concept of a minimum RSS. I think that, in order not to confuse the reader, the values on the axis of the B0 coefficients have been changed to correspond with the text. The axes on the plots below are unaltered.

In [ ]:
# Regression coefficients (Ordinary Least Squares)
regr = LinearRegression()

X = scale(advertising_df.TV, with_mean=True, with_std=False).reshape(-1, 1)
y = advertising_df.Sales

regr.fit(X, y)
print(regr.intercept_)
print(regr.coef_)

In [ ]:
# Create grid coordinates for plotting
B0 = np.linspace(regr.intercept_ - 2, regr.intercept_ + 2, 50)
B1 = np.linspace(regr.coef_ - 0.02, regr.coef_ + 0.02, 50)
xx, yy = np.meshgrid(B0, B1, indexing="xy")
Z = np.zeros((B0.size, B1.size))

# Calculate Z-values (RSS) based on grid of coefficients
for (i, j), v in np.ndenumerate(Z):
    Z[i, j] = ((y - (xx[i, j] + X.ravel() * yy[i, j])) ** 2).sum() / 1000

# Minimized RSS
min_RSS = r"$\beta_0$, $\beta_1$ for minimized RSS"
min_rss = (
    np.sum((regr.intercept_ + regr.coef_ * X - y.values.reshape(-1, 1)) ** 2) / 1000
)
min_rss

In [ ]:
fig = plt.figure(figsize=(15, 6))
fig.suptitle("RSS - Regression coefficients", fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection="3d")

# Left plot
CS = ax1.contour(xx, yy, Z, cmap=plt.cm.Set1, levels=[2.15, 2.2, 2.3, 2.5, 3])
ax1.scatter(regr.intercept_, regr.coef_[0], c="r", label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")

# Right plot
ax2.plot_surface(xx, yy, Z, rstride=3, cstride=3, alpha=0.3)
ax2.contour(
    xx,
    yy,
    Z,
    zdir="z",
    offset=Z.min(),
    cmap=plt.cm.Set1,
    alpha=0.4,
    levels=[2.15, 2.2, 2.3, 2.5, 3],
)
ax2.scatter3D(regr.intercept_, regr.coef_[0], min_rss, c="r", label=min_RSS)
ax2.set_zlabel("RSS")
ax2.set_zlim(Z.min(), Z.max())
ax2.set_ylim(0.02, 0.07)

# settings common to both plots
for ax in fig.axes:
    ax.set_xlabel(r"$\beta_0$", fontsize=17)
    ax.set_ylabel(r"$\beta_1$", fontsize=17)
    ax.set_yticks([0.03, 0.04, 0.05, 0.06])
    ax.legend()

### Confidence interval on page 67 & Table 3.1 & 3.2 - Statsmodels 

In [ ]:
est = ols("Sales ~ TV", advertising_df).fit()
est.summary().tables[1]

In [ ]:
# RSS with regression coefficients
(
    (advertising_df.Sales - (est.params[0] + est.params[1] * advertising_df.TV)) ** 2
).sum() / 1000

### Table 3.1 & 3.2 - Scikit-learn

In [ ]:
regr = LinearRegression()

X = advertising_df.TV.values.reshape(-1, 1)
y = advertising_df.Sales

regr.fit(X, y)
print(regr.intercept_)
print(regr.coef_)

In [ ]:
Sales_pred = regr.predict(X)
r2_score(y, Sales_pred)